In [1]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json

Initialize Mongodb database

In [2]:
client = MongoClient()

In [3]:
db = client['philadelphia_home_prices']

In [4]:
db.list_collection_names()

['prices_messy']

In [5]:
#uncomment to drop collection if needed during testing
db.drop_collection('prices_messy')

{'nIndexesWas': 1, 'ns': 'philadelphia_home_prices.prices_messy', 'ok': 1.0}

In [6]:
col = db.create_collection('prices_messy')

In [7]:
db.list_collection_names()

['prices_messy']

Add old data

In [8]:
df = pd.read_csv('house_data.csv')#.set_index(['Address'])
df2 = pd.read_csv('house_data_day2.csv')
df = pd.concat([df,df2])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 702 entries, 0 to 504
Data columns (total 26 columns):
Unnamed: 0             702 non-null int64
Address                702 non-null object
Locality               702 non-null object
Region                 702 non-null object
Postal Code            702 non-null int64
Price                  702 non-null object
SQFT House             702 non-null object
SQFT Lot               702 non-null object
Full Baths             702 non-null object
Half Baths             702 non-null object
Bedrooms               702 non-null object
Garage                 702 non-null object
Master Bath            702 non-null object
Cooling                702 non-null object
Heating                702 non-null object
Elementary School      702 non-null object
Middle School          702 non-null object
High School            702 non-null object
School District        702 non-null object
Annual Tax             702 non-null object
House Type             702 non-null o

In [10]:
df.head()

,Unnamed: 0,Address,Locality,Region,Postal Code,Price,SQFT House,SQFT Lot,Full Baths,Half Baths,...,Middle School,High School,School District,Annual Tax,House Type,Build Year,Updated,Neighborhood,Association Monthly,Schoool District
0,0,724 W Chelten Ave,Philadelphia,PA,19126,459000,3290,17689,3,3,...,[],"['High School: CENTRAL ', 'High School: CENTRAL']",School District: THE SCHOOL DISTRICT OF PHILAD...,4205,Single family,1925,05/15/2020,['Neighborhood: COBBS CREEK<'],['0'],53.0
1,1,2230 S Carlisle St,Philadelphia,PA,19145,250000,1050,689,1,[''],...,[],[],School District: THE SCHOOL DISTRICT OF PHILAD...,1407,Multi family,1920,05/16/2020,['Neighborhood: COBBS CREEK<'],['0'],53.0
2,2,1247 E Columbia Ave Apt 30,Philadelphia,PA,19125,500000,1092,[],2,1,...,[],[],School District: THE SCHOOL DISTRICT OF PHILAD...,897,Condo,2016,05/17/2020,['Neighborhood: COBBS CREEK<'],['312'],53.0
3,3,500 Admirals Way Apt 112,Philadelphia,PA,19146,383500,902,[],1,[''],...,[],[],School District: THE SCHOOL DISTRICT OF PHILAD...,4878,Condo,2006,05/17/2020,['Neighborhood: COBBS CREEK<'],['432'],53.0
4,4,1021 Hazelwood Dr,Philadelphia,PA,19150,370000,2513,2716,3,1,...,[],"['High School: ROXBOROUGH ', 'High School: ROX...",School District: THE SCHOOL DISTRICT OF PHILAD...,720,Single family,2016,05/15/2020,['Neighborhood: COBBS CREEK<'],['0'],53.0


In [11]:
df = df.drop(['Unnamed: 0'],1)

In [12]:
df.head(1)

,Address,Locality,Region,Postal Code,Price,SQFT House,SQFT Lot,Full Baths,Half Baths,Bedrooms,...,Middle School,High School,School District,Annual Tax,House Type,Build Year,Updated,Neighborhood,Association Monthly,Schoool District
0,724 W Chelten Ave,Philadelphia,PA,19126,459000,3290,17689,3,3,5,...,[],"['High School: CENTRAL ', 'High School: CENTRAL']",School District: THE SCHOOL DISTRICT OF PHILAD...,4205,Single family,1925,05/15/2020,['Neighborhood: COBBS CREEK<'],['0'],53.0


In [13]:
df.reset_index(inplace=True)

In [14]:
df.head(1)

,index,Address,Locality,Region,Postal Code,Price,SQFT House,SQFT Lot,Full Baths,Half Baths,...,Middle School,High School,School District,Annual Tax,House Type,Build Year,Updated,Neighborhood,Association Monthly,Schoool District
0,0,724 W Chelten Ave,Philadelphia,PA,19126,459000,3290,17689,3,3,...,[],"['High School: CENTRAL ', 'High School: CENTRAL']",School District: THE SCHOOL DISTRICT OF PHILAD...,4205,Single family,1925,05/15/2020,['Neighborhood: COBBS CREEK<'],['0'],53.0


In [15]:
prices_dict = df.to_json(orient="index")
parsed = json.loads(prices_dict)
list_of_dics = [value for value in parsed.values()]

In [16]:
len(list_of_dics)

702

In [17]:
col.insert_many(list_of_dics)

In [18]:
col.count()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


702

In [19]:
list_of_dics[0]

{'index': 0,
 'Address': '724 W Chelten Ave',
 'Locality': 'Philadelphia',
 'Region': 'PA',
 'Postal Code': 19126,
 'Price': '459000',
 'SQFT House': '3290',
 'SQFT Lot': '17689',
 'Full Baths': '3',
 'Half Baths': '3',
 'Bedrooms': '5',
 'Garage': '[]',
 'Master Bath': '[]',
 'Cooling': "['Whole House Fan']",
 'Heating': "['Hot Water']",
 'Elementary School': '[]',
 'Middle School': '[]',
 'High School': "['High School: CENTRAL ', 'High School: CENTRAL']",
 'School District': 'School District: THE SCHOOL DISTRICT OF PHILADELPHIA ',
 'Annual Tax': '4205',
 'House Type': 'Single family',
 'Build Year': '1925',
 'Updated': '05/15/2020',
 'Neighborhood': "['Neighborhood: COBBS CREEK<']",
 'Association Monthly': "['0']",
 'Schoool District': 53.0,
 '_id': ObjectId('5f8f3b5c10b8d8dd650e2938')}

In [20]:
len(list_of_dics)

702

In [21]:
db.collection_names()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['prices_messy', 'listing_urls', 'listing_data']

In [22]:
#pull urls from mongo db
cursor = db.listing_data.find({})
listings = list(cursor)

In [23]:
listings

[{'_id': ObjectId('5f9a0001329f96e5db63d260'),
  'Address': '1232 N Mascher St Unit A,',
  'Locality': 'Philadelphia',
  'Region': 'PA',
  'Postal Code': '19122',
  'Price': ['639,900',
   '639,900',
   '639,900',
   '639,900',
   '639,900',
   '639,900',
   '639,900',
   '639,900',
   '639,900'],
  'Days on Market': ['37'],
  'SQFT House': ['3,188', '3,188', '3,188', '3,188'],
  'SQFT Lot': ['960', '960'],
  'Full Baths': ['3'],
  'Half Baths': [],
  'Bedrooms': ['3'],
  'Garage': [' 1'],
  'Master Bath': [],
  'Cooling': [' Central A/C'],
  'Heating': [' Forced Air Heating'],
  'Annual Tax': ['2056'],
  'House Type': [' End of Row/Townhouse'],
  'Build Year': [' 2015', ' 2015'],
  'Updated': [' 08/07/2020'],
  'Neighborhood': ['OLD KENSINGTON'],
  'Neighborhood median sales': ['250,000'],
  'Neighborhood Median DOM': ['63'],
  'Neighborhood Price SQFT': ['195'],
  'Association': ['Yes'],
  'Association Monthly': ['172']},
 {'_id': ObjectId('5f9a00b1329f96e5db63d261'),
  'Address': '2